In [1]:
import time
import urllib.request
from urllib.request import urlopen
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import random
from urllib.error import HTTPError
from selenium.common.exceptions import NoSuchElementException
import re
import numpy as np
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pymysql
import os
from openpyxl import Workbook
from pyvirtualdisplay import Display

import matplotlib.pyplot as plt
import pandas as pd
import sys
import random

In [37]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    link_list = []
    
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)
                    
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height != last_height:
            soup = BeautifulSoup(driver.page_source,'html.parser')
            driver.findElement(By.xpath("//a[@href='/p/B67fmKDh5c5/']")).click();

        else:
            # If heights are the same it will exit the function
            break           
            
        last_height = new_height        

In [38]:
def get_data(tag):
    options = Options()
    # chrome을 전체화면으로 넓히는 옵션입니다.
    options.add_argument('--start-fullscreen')

    #display = Display(visible=0, size=(1024,768))
    #sdisplay.start()
    username = "suuu_0829"
    password = "rla132489"

    driver = webdriver.Chrome(executable_path='./chromedriver')
    getdriver = ("https://www.instagram.com/accounts/login/")
    driver.get(getdriver)

    time.sleep(2)

    id_input = driver.find_elements_by_css_selector('#react-root > section > main > div > article > div > div > div > form > div > div > label > input')[0]
    id_input.send_keys(username)
    password_input = driver.find_elements_by_css_selector('#react-root > section > main > div > article > div > div > div > form > div > div > label > input')[1]
    password_input.send_keys(password)

    password_input.submit()

    time.sleep(3)

    chrome_options = Options()

    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-setuid-sandbox")


    driver.get('https://www.instagram.com/explore/tags/'+tag+'/')

    time.sleep(3)
    
    links = scroll(driver,2)
    links = list(set(scroll(driver,2)))
    
    print(len(links))
    
    t = random.randrange(0,3)
    
    collect_driver = webdriver.Chrome(executable_path='./chromedriver')
    
    post_num = 0
    data = []
    
    for i in range(len(links)):
        file_data = {}
        try:
            collect_driver.implicitly_wait(3)
            #페이지 열기
            collect_driver.get('https://www.instagram.com' + (links[i]))
            #페이지 소스 가져오기
            collect_soup = BeautifulSoup(collect_driver.page_source,'html.parser')
            #게시글 작성자 이름
            user_id = collect_driver.find_element_by_class_name('_6lAjh').text
            file_data['id'] = user_id

            #좋아요 개수
            like = collect_driver.find_element_by_class_name('Nm9Fw').text
            like_count = like.split(' ')[1]
            file_data['like'] = like_count

            #게시글 생성날짜
            post_date = collect_soup.find('time',{'class':'_1o9PC Nzb55'}).get('datetime')  
            file_data['date'] = post_date[0:10]

            #해시태그
            tag_source = collect_soup.find_all('meta',{'property':'instapp:hashtags'})
            tag_list = []
            for j in range(len(tag_source)):
                tag_list.append(tag_source[j].get('content'))
            tag_string = "|".join(tag_list)
            file_data['hashtags'] = tag_string

            #댓글
            comment_list = []
            comment_source = collect_soup.find_all('div',{'class':'C4VMK'})
            for k in range(1,len(comment_source)):
                comment = comment_source[k].find('span').text
                #comment = comment.replace("'",";")
                c_id = comment_source[k].find('a').text
                comment_list.append([comment,c_id])
            file_data['comments'] = comment_list

            #텍스트 본문
            try:
                text_source = collect_soup.find('div',{'class':'C4VMK'}).find('span').text
            except:
                text_source = ""
            finally:
                if str(text_source) == "인증됨":
                    text_source = collect_soup.find('div',{'class':'C4VMK'}).find_all('span')[1].text            

            file_data['text'] = text_source

            data.append(file_data)

        except NoSuchElementException:
            print(post_num,"번째 페이지 삭제됨")
            continue

        if(post_date[0:10] == '2019-04-07'):
            collect_driver.close()
            return data
        post_num +=1
        time.sleep(t)
                               
    return data

In [39]:
def make_df(data):    
    user_id = []
    like = []
    date = []
    hashtags = []
    comments = []
    post = []

    # id, like, date, hashtags, comments, text

    for i in range(len(data)):
        user_id.append(str(data[i]['id']))
        like.append(str(data[i]['like']))
        date.append(str(data[i]['date']))
        hashtags.append(str(data[i]['hashtags']))
        comments.append(str(data[i]['comments']))
        post.append(str(data[i]['text']))

    result = pd.DataFrame({'id':user_id, 'like':like, 'date':date, 'hashtags':hashtags, 'comments':comments, 'post':post})

    result.to_csv("instagram.csv", mode='w')

In [40]:
def main(tag):
    data = get_data(tag)
    print(len(data))
    print("=======================")
    make_df(data)

In [41]:
if __name__ == "__main__":
    # execute only if run as a script
    tag = 'parasitemovie'
    main(tag)

0
0
